In [2]:
import pandas as pd
from example.discretizer.discretizer_d0  import AVDiscretizer
from example.discretizer.discretizer_d1  import AVDiscretizerD1
from example.discretizer.discretizer_d2  import AVDiscretizerD2
from example.discretizer.utils import Action
import matplotlib.pyplot as plt
from pathlib import Path
import pgeon.policy_graph as PG
from example.environment import SelfDrivingEnvironment
from nuscenes import NuScenes

In [3]:
#path where you stored the copy of the nuScenes dataset.
DATAROOT = Path('/home/saramontese/Desktop/MasterThesis/example/dataset/data/sets/nuscenes')

#MINI
nuscenes = NuScenes('v1.0-mini', dataroot=DATAROOT)

#FULL
#nuscenes = NuScenes('v1.0-trainval', dataroot=DATAROOT)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.454 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


The dataset contains the following columns for each state of the self-driving car:

- instance_token: A unique identifier for the vehicle instance.
- translation: The vehicle's position in 3D space (x, y, z coordinates).
- yaw: The orientation of the vehicle around the vertical axis (rotation angle).
- velocity: The vehicle's speed in a given direction.
- acceleration: The change in velocity over time.
- heading_change_rate: The rate of change of the vehicle's direction.
- timestamp: The time at which the state was recorded.
- scene_token: A unique identifier for the scenario or environment the vehicle is in
- detected_objects: detected objects in the surrounding of the vehicle


Load the CSV data into a DataFrame


In [5]:

dtype_dict = {
    'modality': 'category',  # for limited set of modalities, 'category' is efficient
    'scene_token': 'str',  
    'timestamp': 'str',  # To enable datetime operations
    'rotation': 'object',  # Quaternion (lists)
    'x': 'float64',
    'y': 'float64',
    'z': 'float64',
    'yaw': 'float64',  
    'velocity': 'float64',
    'acceleration': 'float64',
    'heading_change_rate': 'float64',
    'delta_local_x': 'float64',
    'delta_local_y': 'float64'}
df = pd.read_csv(DATAROOT / 'full_v1.0-trainval_lidar_1_10.csv', dtype=dtype_dict, parse_dates=['timestamp'])
city ='singapore-onenorth'  #'boston-seaport'#boston-seaport'#
df = df[df['location'] == city]
#df['detect_CAM_FRONT'] = df['detect_CAM_FRONT'].apply(lambda x: ast.literal_eval(x))
#df['detect_CAM_BACK'] = df['detect_CAM_BACK'].apply(lambda x: ast.literal_eval(x))
df.head()

,sample_token,scene_token,steering_angle,location,night,rain,CAM_FRONT,modality,timestamp,rotation,x,y,z,yaw,velocity,yaw_rate,acceleration
40,1a229d68b67e467c8189c7bf492b5a2d,00590cbfa24a430a8c274b51e1c71231,1.404990,singapore-onenorth,0,0,{},lidar,2018-08-01 08:39:22.196166,"[0.11307311013999582, 0.009654392942181369, -0...",334.774028,975.531223,0.0,2.915012,1.272117,0.038479,0.000000
41,bb5b65334c594f3f94b54137a1f021a4,00590cbfa24a430a8c274b51e1c71231,1.404990,singapore-onenorth,0,0,{},lidar,2018-08-01 08:39:22.696597,"[0.10350989238658959, 0.00968827228603952, -0....",334.153555,975.673638,0.0,2.934268,1.272117,0.038479,0.000000
42,31094689d26b4fbeb8bb4938d0f3c6dd,00590cbfa24a430a8c274b51e1c71231,1.404990,singapore-onenorth,0,0,{},lidar,2018-08-01 08:39:23.146716,"[0.09485416033489596, 0.010046756429239528, -0...",333.634601,975.784381,0.0,2.951691,1.178886,0.038709,-0.207124
43,34411ab5467041d797df0a32b766acaa,00590cbfa24a430a8c274b51e1c71231,1.404990,singapore-onenorth,0,0,{},lidar,2018-08-01 08:39:23.647156,"[0.08532914510578349, 0.011252554942682395, -0...",333.090040,975.891821,0.0,2.970841,1.109141,0.038266,-0.139367
44,1062a0889bc344879c71cede2f6fe8cb,00590cbfa24a430a8c274b51e1c71231,1.411971,singapore-onenorth,0,0,{},lidar,2018-08-01 08:39:24.147583,"[0.07666723910993856, 0.012305012220579333, -0...",332.597577,975.978516,0.0,2.988235,0.999217,0.034759,-0.219661


## Trajectory of AV in a Scene

Let's do an example of computing trajectory of a vehicle in a scene. We then check if the rendering of the scene match the computed trajectory.

List of scenes

In [6]:
df['scene_token'].unique()

array(['00590cbfa24a430a8c274b51e1c71231',
       '01c8c59260db4a3682d7b4f8da65425e',
       '02e484442a2f411b971edcf96ebbe885',
       '03ee880dd4e348f4b3407f0d073c7c70',
       '04061484361145f5ac6f90a5103b84a6',
       '080a52cb8f59489b9cddc7b721808088',
       '095d9b93b583425f910ae2afaf1d017d',
       '0be1ff07a8f148ca9535fb7f0deaf828',
       '0c601ff2bf004fccafec366b08bf29e2',
       '0cc28fe2c1064fb9a51f1647c28ca564',
       '0e37d4a357db4246a908cfd97d17efc6',
       '105c227c304b4cb18e98a5be6b2d8302',
       '10dfdae842b543938a230f25a1e9f934',
       '11058d54abcd4829828523040372ce05',
       '12cd5f88732749a58725528bb3d86958',
       '1319a9c0288f4ff9ae32d3f33b3bae17',
       '17302a41218442ffbb0b094adb0669ab',
       '1aa633b683174280b243e0a9a7ad9171',
       '1c89941a6935484182ca19eddcd3bc77',
       '1d4db80d13f342aba4881b38099bc4b7',
       '1d914f73a4a243c3acac50d24f083aac',
       '1fa63d31aaa7459383b4414715cd12db',
       '1fbcc26ebf6948bb964d3ae74939e8ea',
       '201

Select a scene and test the  algorithm


In [7]:
scene_test = df['scene_token'].unique()[0]
example_scene_df = df[df['scene_token']==scene_test]
example_scene_df.head()

,sample_token,scene_token,steering_angle,location,night,rain,CAM_FRONT,modality,timestamp,rotation,x,y,z,yaw,velocity,yaw_rate,acceleration
40,1a229d68b67e467c8189c7bf492b5a2d,00590cbfa24a430a8c274b51e1c71231,1.404990,singapore-onenorth,0,0,{},lidar,2018-08-01 08:39:22.196166,"[0.11307311013999582, 0.009654392942181369, -0...",334.774028,975.531223,0.0,2.915012,1.272117,0.038479,0.000000
41,bb5b65334c594f3f94b54137a1f021a4,00590cbfa24a430a8c274b51e1c71231,1.404990,singapore-onenorth,0,0,{},lidar,2018-08-01 08:39:22.696597,"[0.10350989238658959, 0.00968827228603952, -0....",334.153555,975.673638,0.0,2.934268,1.272117,0.038479,0.000000
42,31094689d26b4fbeb8bb4938d0f3c6dd,00590cbfa24a430a8c274b51e1c71231,1.404990,singapore-onenorth,0,0,{},lidar,2018-08-01 08:39:23.146716,"[0.09485416033489596, 0.010046756429239528, -0...",333.634601,975.784381,0.0,2.951691,1.178886,0.038709,-0.207124
43,34411ab5467041d797df0a32b766acaa,00590cbfa24a430a8c274b51e1c71231,1.404990,singapore-onenorth,0,0,{},lidar,2018-08-01 08:39:23.647156,"[0.08532914510578349, 0.011252554942682395, -0...",333.090040,975.891821,0.0,2.970841,1.109141,0.038266,-0.139367
44,1062a0889bc344879c71cede2f6fe8cb,00590cbfa24a430a8c274b51e1c71231,1.411971,singapore-onenorth,0,0,{},lidar,2018-08-01 08:39:24.147583,"[0.07666723910993856, 0.012305012220579333, -0...",332.597577,975.978516,0.0,2.988235,0.999217,0.034759,-0.219661


Compute the trajectoy of the scene

In [8]:
env = SelfDrivingEnvironment(city,dataroot=DATAROOT)
av_discretizer = AVDiscretizerD1(env)
pg = PG.PolicyGraph(env, av_discretizer)
pg = pg.fit(df, update=False, verbose=True)

Fitting PG from scenes...:   0%|          | 0/183 [00:00<?, ?it/s]

scene token: 00590cbfa24a430a8c274b51e1c71231


Fitting PG from scenes...:   1%|          | 1/183 [00:06<20:43,  6.83s/it]

frame 0 --> [PedestrianNearby(NO), IsTwoWheelNearby(NO), BlockProgress(END), LanePosition(ALIGNED), NextIntersection(LEFT), Velocity(MOVING), Rotation(LEFT), StopAreaNearby(TURN_STOP), IsZebraNearby(NO), IsTrafficLightNearby(NO), FrontObjects(NO)]
action: Action.TURN_LEFT
frame 1 --> [PedestrianNearby(NO), IsTwoWheelNearby(NO), BlockProgress(END), LanePosition(ALIGNED), NextIntersection(LEFT), Velocity(MOVING), Rotation(LEFT), StopAreaNearby(TURN_STOP), IsZebraNearby(NO), IsTrafficLightNearby(NO), FrontObjects(NO)]
action: Action.TURN_LEFT
frame 2 --> [PedestrianNearby(NO), IsTwoWheelNearby(NO), BlockProgress(END), LanePosition(ALIGNED), NextIntersection(LEFT), Velocity(MOVING), Rotation(LEFT), StopAreaNearby(TURN_STOP), IsZebraNearby(NO), IsTrafficLightNearby(NO), FrontObjects(NO)]
action: Action.TURN_LEFT
frame 3 --> [PedestrianNearby(NO), IsTwoWheelNearby(NO), BlockProgress(END), LanePosition(ALIGNED), NextIntersection(LEFT), Velocity(MOVING), Rotation(LEFT), StopAreaNearby(TURN_STO

Fitting PG from scenes...:   1%|          | 2/183 [00:13<19:54,  6.60s/it]

frame 0 --> [PedestrianNearby(NO), IsTwoWheelNearby(NO), BlockProgress(START), LanePosition(ALIGNED), NextIntersection(STRAIGHT), Velocity(MOVING), Rotation(RIGHT), StopAreaNearby(NO), IsZebraNearby(NO), IsTrafficLightNearby(NO), FrontObjects(NO)]
action: Action.TURN_RIGHT
frame 1 --> [PedestrianNearby(NO), IsTwoWheelNearby(NO), BlockProgress(MIDDLE), LanePosition(ALIGNED), NextIntersection(STRAIGHT), Velocity(MOVING), Rotation(RIGHT), StopAreaNearby(NO), IsZebraNearby(NO), IsTrafficLightNearby(NO), FrontObjects(NO)]
action: Action.STRAIGHT
frame 2 --> [PedestrianNearby(NO), IsTwoWheelNearby(NO), BlockProgress(MIDDLE), LanePosition(ALIGNED), NextIntersection(STRAIGHT), Velocity(MOVING), Rotation(FORWARD), StopAreaNearby(NO), IsZebraNearby(NO), IsTrafficLightNearby(NO), FrontObjects(NO)]
action: Action.GAS
frame 3 --> [PedestrianNearby(NO), IsTwoWheelNearby(NO), BlockProgress(MIDDLE), LanePosition(ALIGNED), NextIntersection(STRAIGHT), Velocity(MOVING), Rotation(FORWARD), StopAreaNearby(

Fitting PG from scenes...:   2%|▏         | 3/183 [00:19<19:26,  6.48s/it]

frame 0 --> [PedestrianNearby(NO), IsTwoWheelNearby(YES), BlockProgress(END), LanePosition(CENTER), NextIntersection(STRAIGHT), Velocity(STOPPED), Rotation(FORWARD), StopAreaNearby(NO), IsZebraNearby(NO), IsTrafficLightNearby(YES), FrontObjects(YES)]
action: Action.IDLE
frame 1 --> [PedestrianNearby(NO), IsTwoWheelNearby(YES), BlockProgress(END), LanePosition(CENTER), NextIntersection(STRAIGHT), Velocity(STOPPED), Rotation(FORWARD), StopAreaNearby(NO), IsZebraNearby(NO), IsTrafficLightNearby(YES), FrontObjects(YES)]
action: Action.IDLE
frame 2 --> [PedestrianNearby(NO), IsTwoWheelNearby(YES), BlockProgress(END), LanePosition(CENTER), NextIntersection(STRAIGHT), Velocity(STOPPED), Rotation(FORWARD), StopAreaNearby(NO), IsZebraNearby(NO), IsTrafficLightNearby(YES), FrontObjects(YES)]
action: Action.IDLE
frame 3 --> [PedestrianNearby(NO), IsTwoWheelNearby(YES), BlockProgress(END), LanePosition(CENTER), NextIntersection(STRAIGHT), Velocity(STOPPED), Rotation(FORWARD), StopAreaNearby(NO), I

/home/saramontese/.local/lib/python3.8/site-packages/nuscenes/map_expansion/map_api.py:1887: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if new_polygon.geom_type is 'Polygon':
/home/saramontese/.local/lib/python3.8/site-packages/nuscenes/map_expansion/map_api.py:1924: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if layer_name is 'traffic_light':
/home/saramontese/.local/lib/python3.8/site-packages/nuscenes/map_expansion/map_api.py:1970: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if new_polygon.geom_type is 'Polygon':
/home/saramontese/.local/lib/python3.8/site-packages/nuscenes/map_expansion/map_api.py:1985: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if new_polygon.geom_type is 'Polygon':
/home/saramontese/.local/lib/python3.8/site-packages/nuscenes/map_expansion/map_api.py:2005: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if layer_name is 'traffic_light':
/home/saramontese/.local/lib/python3.8/site-packages/nuscenes/map

KeyboardInterrupt: 

To render the scene (video):

In [30]:
#nuscenes.render_scene(scene_test)

In [10]:
environment = SelfDrivingEnvironment(city='all')
discretizer = AVDiscretizerD1(environment, obj_discretization='binary', vel_discretization='multiple')
nodes_path = f'example/dataset/data/policy_graphs_best/PG_trainval_Call_D1b_Wall_Tall_nodes.csv'
edges_path = f'example/dataset/data/policy_graphs_best/PG_trainval_Call_D1b_Wall_Tall_edges.csv'
pg = PG.PolicyGraph.from_nodes_and_edges(nodes_path, edges_path, environment, discretizer)


# Policy Graph Generation

In [11]:
print(f'Number of nodes: {len(pg.nodes)}')
print(f'Number of edges: {len(pg.edges)}')

Number of nodes: 2153
Number of edges: 10285


In [12]:
arbitrary_state = list(pg.nodes)[9]

print(arbitrary_state)
print(f'  Times visited: {pg.nodes[arbitrary_state]["frequency"]}')
print(f'  p(s):          {pg.nodes[arbitrary_state]["probability"]:.3f}')

(PedestrianNearby(NO), IsTwoWheelNearby(NO), BlockProgress(START), LanePosition(ALIGNED), NextIntersection(NONE), Velocity(LOW), Rotation(RIGHT), StopAreaNearby(NO), IsZebraNearby(NO), IsTrafficLightNearby(NO), FrontObjects(YES))
  Times visited: 34
  p(s):          0.001


In [13]:
arbitrary_edge = list(pg.edges)[20]

print(f'From:    {arbitrary_edge[0]}')
print(f'Action:  {arbitrary_edge[2]}')
print(f'To:      {arbitrary_edge[1]}')
print(f'  Times visited:      {pg[arbitrary_edge[0]][arbitrary_edge[1]][arbitrary_edge[2]]["frequency"]}')
print(f'  p(s_to,a | s_from): {pg[arbitrary_edge[0]][arbitrary_edge[1]][arbitrary_edge[2]]["probability"]:.3f}')

Fitting PG from scenes...:   2%|▏         | 3/183 [01:02<1:02:22, 20.79s/it]

From:    (PedestrianNearby(NO), IsTwoWheelNearby(NO), BlockProgress(START), LanePosition(ALIGNED), NextIntersection(NONE), Velocity(LOW), Rotation(FORWARD), StopAreaNearby(NO), IsZebraNearby(NO), IsTrafficLightNearby(NO), FrontObjects(YES))
Action:  5
To:      (PedestrianNearby(NO), IsTwoWheelNearby(NO), BlockProgress(START), LanePosition(ALIGNED), NextIntersection(NONE), Velocity(LOW), Rotation(FORWARD), StopAreaNearby(NO), IsZebraNearby(NO), IsTrafficLightNearby(NO), FrontObjects(NO))
  Times visited:      2
  p(s_to,a | s_from): 0.013


### Using PG-based policies

In [21]:
from pgeon.policy_graph import PGBasedPolicy, PGBasedPolicyMode, PGBasedPolicyNodeNotFoundMode

from example.discretizer.utils import PedestrianNearby, BlockProgress, IsTwoWheelNearby, IsTrafficLightNearby,IsZebraNearby,StopAreaNearby, FrontObjects, Action, LanePosition, BlockProgress, NextIntersection, Velocity, Rotation

from pgeon.discretizer import  Predicate

In [16]:
policy_mode = PGBasedPolicyMode.STOCHASTIC
node_not_found_mode = PGBasedPolicyNodeNotFoundMode.FIND_SIMILAR_NODES

policy = PGBasedPolicy(pg, policy_mode,node_not_found_mode )
pg._normalize()
print(f'Policy Graph with {policy_mode} and {node_not_found_mode}')
print(f'PG number of nodes: {len(policy.pg.nodes)}')
print(f'PG number of edges: {len(policy.pg.edges)}')
        


initial_state = list(pg.nodes)[1]  
#final_state = list(pg.nodes)[5]
action = policy.act(initial_state)

action



Policy Graph with PGBasedPolicyMode.STOCHASTIC and PGBasedPolicyNodeNotFoundMode.FIND_SIMILAR_NODES
PG number of nodes: 2153
PG number of edges: 10285


4

In [17]:
str_obs = discretizer.state_to_str(initial_state)
str_obs

'PedestrianNearby(NO) IsTwoWheelNearby(NO) BlockProgress(START) LanePosition(ALIGNED) NextIntersection(NONE) Velocity(LOW) Rotation(FORWARD) StopAreaNearby(NO) IsZebraNearby(NO) IsTrafficLightNearby(NO) FrontObjects(YES)'

Question 1

In [18]:
possible_actions = pg.question1(initial_state, verbose=True)

I will take one of these actions:
	-> GAS 	Prob: 32.67 %
	-> STRAIGHT 	Prob: 30.67 %
	-> BRAKE 	Prob: 30.67 %
	-> GAS_TURN_RIGHT 	Prob: 1.33 %
	-> BRAKE_TURN_LEFT 	Prob: 1.33 %
	-> BRAKE_TURN_RIGHT 	Prob: 1.33 %
	-> IDLE 	Prob: 0.67 %
	-> GAS_TURN_LEFT 	Prob: 0.67 %
	-> TURN_RIGHT 	Prob: 0.67 %


Question 2

In [19]:
action_id = AVDiscretizer.get_action_id(Action.BRAKE)
best_states = pg.question2(action_id, verbose=True)

*********************************
* When do you perform action 5?
*********************************
Most probable in 324 states:
	-> (PedestrianNearby(NO), IsTwoWheelNearby(NO), BlockProgress(START), LanePosition(ALIGNED), NextIntersection(NONE), Velocity(HIGH), Rotation(FORWARD), StopAreaNearby(NO), IsZebraNearby(NO), IsTrafficLightNearby(YES), FrontObjects(NO))
	-> (PedestrianNearby(NO), IsTwoWheelNearby(YES), BlockProgress(MIDDLE), LanePosition(ALIGNED), NextIntersection(NONE), Velocity(LOW), Rotation(FORWARD), StopAreaNearby(NO), IsZebraNearby(YES), IsTrafficLightNearby(YES), FrontObjects(YES))
	-> (PedestrianNearby(NO), IsTwoWheelNearby(NO), BlockProgress(END), LanePosition(CENTER), NextIntersection(NONE), Velocity(MEDIUM), Rotation(FORWARD), StopAreaNearby(TURN_STOP), IsZebraNearby(NO), IsTrafficLightNearby(NO), FrontObjects(NO))
	-> (PedestrianNearby(NO), IsTwoWheelNearby(NO), BlockProgress(END), LanePosition(CENTER), NextIntersection(NONE), Velocity(LOW), Rotation(FORWARD), Sto

Question 3

In [23]:
state = tuple(( Predicate(PedestrianNearby, [PedestrianNearby(1, 'binary')]),
                Predicate(IsTwoWheelNearby, [IsTwoWheelNearby.YES]),
                Predicate(BlockProgress, [BlockProgress.END]),
                
                Predicate(LanePosition, [LanePosition.ALIGNED]),
                Predicate(NextIntersection, [NextIntersection.STRAIGHT]),
                Predicate(Velocity, [Velocity.STOPPED]),
                Predicate(Rotation, [Rotation.FORWARD]),
                Predicate(StopAreaNearby, [StopAreaNearby.NO]), 
                Predicate(IsZebraNearby,[IsZebraNearby.NO]),
                Predicate(IsTrafficLightNearby, [IsTrafficLightNearby.NO]),
                Predicate(FrontObjects,[FrontObjects(1, 'binary')])) )
#state_2 = av_discretizer.str_to_state(av_discretizer.state_to_str(state))

action = policy.act(state)

PGBasedPolicyNodeNotFoundMode.FIND_SIMILAR_NODES


In [24]:
policy_mode= PGBasedPolicyMode.GREEDY
node_not_found_mode = PGBasedPolicyNodeNotFoundMode.FIND_SIMILAR_NODES #PGBasedPolicyNodeNotFoundMode.RANDOM_UNIFORM, 
action = Action.BRAKE
action_id = AVDiscretizer.get_action_id(action)

policy_2 = PGBasedPolicy(pg, mode=policy_mode,  node_not_found_mode=node_not_found_mode)
print(f'Policy Graph with {policy_mode} and {node_not_found_mode}')
print()
print(f'Supposing I was in {state}, '
    f'if I did not choose to {action} was due to...')
policy.pg._normalize() #do i need to normalize before this step?
counterfactuals = policy_2.pg.question3(state, action_id, greedy= True, verbose=True)
        

Policy Graph with PGBasedPolicyMode.GREEDY and PGBasedPolicyNodeNotFoundMode.FIND_SIMILAR_NODES

Supposing I was in (PedestrianNearby(YES), IsTwoWheelNearby(YES), BlockProgress(END), LanePosition(ALIGNED), NextIntersection(STRAIGHT), Velocity(STOPPED), Rotation(FORWARD), StopAreaNearby(NO), IsZebraNearby(NO), IsTrafficLightNearby(NO), FrontObjects(YES)), if I did not choose to Action.BRAKE was due to...
***********************************************
* Why did not you perform X action in Y state?
***********************************************
PGBasedPolicyNodeNotFoundMode.RANDOM_UNIFORM
I would have chosen: 1
I would have chosen 5 under the following conditions:
	I don't know where I would have ended up
